In [1]:
import ee
import folium
import pandas as pd

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [17]:
def add_ee_layer(self, ee_image_object, vis_params, name, opacity = 0.5):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
        opacity=opacity,
    ).add_to(self)
    
folium.Map.add_ee_layer = add_ee_layer

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [11]:
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY
S2_CLOUD_PROBABILITY = "COPERNICUS/S2_CLOUD_PROBABILITY"
s2Clouds = ee.ImageCollection(S2_CLOUD_PROBABILITY)


# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
S2_MSI = "COPERNICUS/S2_SR"
s2Sr = ee.ImageCollection(S2_MSI)


In [12]:
# Properties
START_DATE = ee.Date('2019-01-01')
END_DATE = ee.Date('2019-03-01')
MAX_CLOUD_PROBABILITY = 65
rect_dim = (-76.5, 2.0, -74, 4.0)
region = ee.Geometry.Rectangle(rect_dim)

In [14]:
def apply_cloud_mask(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    not_cloud = clouds.lt(MAX_CLOUD_PROBABILITY) # less-than
    return img.updateMask(not_cloud)

# exclude bad data
def apply_edge_mask(s2_img):
    return s2_img.updateMask(s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()));

In [15]:
criteria = ee.Filter.And(ee.Filter.bounds(region), ee.Filter.date(START_DATE, END_DATE))
s2Sr = s2Sr.filter(criteria).map(apply_edge_mask)
s2Clouds = s2Clouds.filter(criteria)

In [28]:
s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(s2Sr, s2Clouds, ee.Filter.equals(leftField  = 'system:index', rightField = 'system:index'))
s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(apply_cloud_mask).median()
rgbVis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}


In [41]:
folium_map = folium.Map((2, -72), zoom_start=7)
desc = f"S2 SR masked at {MAX_CLOUD_PROBABILITY}%"
folium_map.add_ee_layer(s2CloudMasked, rgbVis, desc, opacity=0.8)

In [42]:
# View raw satellite data
folium_map